In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split    # X스플릿
from sklearn.ensemble import RandomForestClassifier     # 랜덤포레스트
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC
from selenium import webdriver as wb
import time
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
from flask import Flask
from flask import request, redirect

# 웹툰데이터 학습

In [2]:
webtoon = pd.read_csv('최종학습데이터.csv', encoding='cp949')

# 제목화 회차 제목을 인덱스로 지정
webtoon.index = [webtoon.iloc[:,0],webtoon.iloc[:,1]]

# 사용하지 않는 열 삭제
webtoon = webtoon.drop('웹툰 제목', axis = 1)
webtoon = webtoon.drop('회차 제목', axis = 1)

# 문제와 정답으로 나누기
webtoon_X = webtoon.drop('정식연재', axis = 1)
webtoon_y = webtoon['정식연재']

# train, test 나누기
X_train,X_test,y_train,y_test = train_test_split(webtoon_X,webtoon_y,
                                random_state = 3,
                                test_size = 0.3)

# 랜덤포레스트 모델 사용
forest = RandomForestClassifier()

# 학습
forest.fit(X_train,y_train)

RandomForestClassifier()

# 댓글 학습

In [3]:
# 학습데이터 불러오기
comment38 = pd.read_csv('댓글예측완료2.csv', encoding='cp949')

X = comment38['댓글'] # 문제
y = comment38['예측라벨'] # 정답
# train, test 분류
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, 
                                                    shuffle=False,random_state=34)

# pipeline : 토큰화, 모델학습
pipe = make_pipeline(CountVectorizer(), LinearSVC())
pipe.fit(X_train, y_train)

C:\Users\SM2116\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\SM2116\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('linearsvc', LinearSVC())])

# 입력한 웹툰URL을 크롤링해주는 함수 정의

In [8]:
def crawling(url_input):
    driver = wb.Chrome()
    titleList = [] # 제목
    titleList2 = [] # 댓글 하나당 제목 하나씩 가져오기 위한 리스트
    date_list = [] # 회차별 날짜
    count_list = [] # 회차별 조회수
    star_list = [] # 회차별 별점
    comment_list = [] # 댓글
    comment_date_list = []
    comment_like_list = []
    comment_hate_list = []
    viewList = [] # 회차제목
    viewList2 = [] # 댓글 하나당 회차제목 하나씩 가져오기 위한 리스트
    roundList = [] # 회차
    roundList2 = [] # 댓글 하나당 회차 하나씩 가져오기 위한 리스트
    star_cnt_list = [] # 별점 참여인원수
    like_list = [] # 회차별 좋아요
    genre_list = [] # 장르
    list1 = [5, 7] # div태그 자식 선택자 순서
    list2 = [1, 3, 5] # td태그 자식선택자 순서
    num = 0
    haha=0

    url=url_input
    driver.get(url)
    time.sleep(1)
    cnt_first = driver.find_element_by_class_name("first")
    cnt_first.click()
    time.sleep(1)

    for page in range(0, 5):  # 다음화 넘어가기 반복문
        soup = bs(driver.page_source,'lxml')
        # 웹툰 제목
        title = soup.select("div.detail")[0].text
        titleList.append(title.split("\t")[0].split("\n")[1].strip())
        # 장르
        genre = soup.select("#content > div.snb > ul > li.on > a")[0].text
        genre_list.append(genre.strip())
        # 회차
        num += 1
        roundList.append(str(num))
        # 회차제목
        view = soup.select("div.tit_area > div.view")[0].text
        viewList.append(view.split("\n")[1].strip())
        # 등록일
        date = soup.select("dd.date")[0].text
        date_list.append(date)
        # 조회수
        count = int(soup.select("dd.date")[1].text)  # 조회수
        count_list.append(count)
        # 별점참여인원
        star_cnt = int(soup.select("#topTotalStarPoint > span.pointTotalPerson > em")[0].text)
        star_cnt_list.append(star_cnt)
        # 좋아요
        like =soup.select("div.u_likeit_module > a > em")[0].text.strip()
        if(len(like)>=4):
            like = like.split(',')
            like = int(like[0]+like[1])
        else :
            like = int(like)
        like_list.append(like)
        # 별점
        star = float(soup.select("#topPointTotalNumber > strong")[0].text)
        star_list.append(star)
        driver.switch_to.window(driver.window_handles[-1]) # 댓글 iprame 시작
        driver.switch_to.frame("commentIframe")
        soups = bs(driver.page_source,"html.parser") # 댓글 iprame 여기까지
        time.sleep(1)
        # 전체 댓글 클릭
        all_click = driver.find_element_by_tag_name("#cbox_module_wai_u_cbox_sort_option_tab2 > span.u_cbox_sort_label")
        all_click.click()
        driver.switch_to.window(driver.window_handles[-1]) 
        driver.switch_to.frame("commentIframe")
        soups = bs(driver.page_source,"html.parser")
        for d in range(4,500): # 댓글 넘기기 반복
            for c in range(0,15): # 한페이지 안의 댓글
                try:
                    driver.switch_to.window(driver.window_handles[-1]) 
                    driver.switch_to.frame("commentIframe")
                    soups = bs(driver.page_source,"html.parser")
                    comment = soups.select("span.u_cbox_contents")[c].text
                    comment_list.append(comment)
                    comment_date = soups.select("span.u_cbox_date")[c].text
                    comment_date_list.append(comment_date)
                    comment_like =soups.select("em.u_cbox_cnt_recomm")[c].text
                    comment_like_list.append(comment_like)
                    comment_hate = soups.select("em.u_cbox_cnt_unrecomm")[c].text
                    comment_hate_list.append(comment_hate)
                    titleList2.append(title.split("\t")[0].split("\n")[1])
                    viewList2.append(view.split("\n")[1])
                    roundList2.append(str(num))
                except:
                    haha+=1
            try:
                driver.switch_to.window(driver.window_handles[-1]) 
                driver.switch_to.frame("commentIframe")
                soups = bs(driver.page_source,"html.parser")
                two_click = driver.find_element_by_tag_name("#cbox_module > div > div.u_cbox_paginate > div > strong + a")
                two_click.click()
    #                             time.sleep(0.5)
            except:
                haha+=1
    #                             if two_click == None: # 만약 더이상 댓글페이지가 없으면
                break # 댓글페이지 넘기기 멈춰
        try:
            driver.switch_to.window(driver.window_handles[-1]) # 페이지 넘어갈때
            next_click = driver.find_element_by_tag_name("div.btn_area > span.next > a")
            next_click.click()
        except:
            haha+=1

    driver.quit()

    dic = {"웹툰 제목": titleList,"회차 제목":viewList,"회차": roundList, "장르":genre_list, "등록일" : date_list, "조회수": count_list, "회차별 별점": star_list, "별점 참여인원수":star_cnt_list, "좋아요수":like_list}
    dic = pd.DataFrame(dic)
    commentDic = {"웹툰제목":titleList2,"회차 제목":viewList2, "회차": roundList2,"댓글": comment_list, "댓글 등록일" : comment_date_list, "댓글 좋아요": comment_like_list, "댓글 싫어요": comment_hate_list}
    commentDic = pd.DataFrame(commentDic)
    
    return dic,commentDic

# 크롤링한 웹툰을 전처리해주는 함수 정의

In [5]:
def preprocess(dic,commentDic):
    dic['조회수 대비 별점참여'] = dic['별점 참여인원수']/dic['조회수']*1000
    dic['조회수 대비 좋아요'] = dic['좋아요수']/dic['조회수']*1000

    dic['에피소드']=0
    dic['옴니버스']=0
    dic['스토리']=0
    dic['일상']=0
    dic['개그']=0
    dic['판타지']=0
    dic['액션']=0
    dic['드라마']=0
    dic['순정']=0
    dic['감성']=0
    dic['스릴러']=0
    dic['시대극']=0
    dic['스포츠']=0

    dic[dic.loc[1,'장르']]=1

    commentDic['회차'] = commentDic['회차'].astype('int')
    dic['회차'] = dic['회차'].astype('int')
    dic['댓글수']=0

    for i in range(0,len(dic)):
        dic.loc[i,'댓글수'] = len(commentDic[commentDic['회차']==i+1])

    X_comment = commentDic['댓글']
    commentDic['긍정부정'] = pipe.predict(X_comment)

    dic['댓글수 대비 긍정댓글'] = 0.0
    dic['댓글수 대비 부정댓글'] = 0.0

    for i in range(1, len(dic)+1): # 회차반복
        for j in range(0,2): # 예측라벨 반복
            bo1 = commentDic['긍정부정'] == j
            bo2 = commentDic['회차'] == i
            bo3 = dic.loc[i-1,'댓글수']!= 0
            if (j == 0) and (len(commentDic[bo1&bo2])!=0) and (bo3):
                dic.loc[i-1,'댓글수 대비 부정댓글']= (len(commentDic[bo1&bo2]))/(dic.loc[i-1,['댓글수']].values)*100
            elif (j == 1) and (len(commentDic[bo1&bo2])!=0) and (bo3):
                dic.loc[i-1,'댓글수 대비 긍정댓글']= (len(commentDic[bo1&bo2]))/(dic.loc[i-1,['댓글수']].values)*100

    # 제목화 회차 제목을 인덱스로 지정
    dic.index = [dic.iloc[:,0],dic.iloc[:,1]]

    # 사용하지 않는 열 삭제
    dic = dic.drop('웹툰 제목', axis = 1)
    dic = dic.drop('회차 제목', axis = 1)
    dic = dic.drop('등록일', axis = 1)
    dic = dic.drop('장르', axis = 1)

    # 컬럼 순서 정렬
    dic = dic[['회차','조회수','회차별 별점','별점 참여인원수','좋아요수','댓글수','조회수 대비 별점참여','조회수 대비 좋아요','댓글수 대비 긍정댓글','댓글수 대비 부정댓글','에피소드','옴니버스','스토리','일상','개그','판타지','액션','드라마','순정','감성','스릴러','시대극','스포츠'
    ]]
    
    return dic

# 정식연재를 예측해주는 함수 정의

In [6]:
def pre(dic):
    result = forest.predict(dic)
    return result

# 플라스크 서버

In [10]:
from flask import Flask
from flask import request, redirect

app = Flask(__name__) 
@app.route('/learn', methods=['GET','POST'])
def index():     
    
    # Webtoon_input.jsp에서 url주소를 받아오기
    if request.method == 'GET':
        url = request.args['url']
        
    else:
        url = request.form['url']
    
    # 크롤링
    crawl = crawling(url)
    dic = crawl[0]
    commentDic = crawl[1]
    
    # 전처리
    dic = preprocess(dic,commentDic)
    
    # 예측
    result = pre(dic)
    
    # 각 회차별 결과값을 변수에 넣기
    vol1 = result[0]
    vol2 = result[1]
    vol3 = result[2]
    vol4 = result[3]
    vol5 = result[4]
        
    url = 'http://localhost:8090/WebToon/Webtoon_input_result.jsp?vol1='+str(vol1)+'&vol2='+str(vol2)+'&vol3='+str(vol3)+'&vol4='+str(vol4)+'&vol5='+str(vol5)
    return redirect(url)

if __name__ == '__main__':
    app.run(host='127.0.0.1', port='5000')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Jul/2021 09:15:20] "GET /learn?url=https%3A%2F%2Fcomic.naver.com%2FbestChallenge%2Flist%3FtitleId%3D761467 HTTP/1.1" 302 -
